## accuracy with rfe

In [31]:
 #-*- coding: utf-8 -*-
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [32]:
def importOwnLib():
    if '/home/miruware/aProjects/lib' not in sys.path:
        sys.path.append('/home/miruware/aProjects/lib')
    else:
        print("lib path is already exists.")

importOwnLib()

import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, train_test_split, cross_val_score
from sklearn.feature_selection import RFECV, RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import sonyalib as sonya
import keras
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, BatchNormalization, Dropout
# from tensorflow.python.keras._impl.keras import Model
# from tensorflow.python.keras._impl.keras.models import Model
from keras import losses
from keras.optimizers import adam
import tensorflow as tf

lib path is already exists.


In [38]:
PATH_EXCEL = './BRC2019_CRF_merged_200923 완료.xlsx'
metadata = sonya.get_metadata(PATH_EXCEL)

df_filtered = metadata.dropna(axis=0, how='any').reset_index(drop=True)  # NaN drop
properties = list(df_filtered.columns.values)
properties.remove('label')
# properties.remove('cN')
properties.remove('ID')
X_origin = df_filtered['cN']
y_origin = df_filtered['label']

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X_origin, y_origin, test_size=0.3, random_state=0)

In [40]:
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr
def mlp_model(num_input, dropout=0, lr=0.005, l1=9, l2=9):
    keras.backend.clear_session()
    ## 모델 구성하기
    model = Sequential()
    # print learning rate
    optimizer = adam(lr=0.001)
    lr_metric = get_lr_metric(optimizer)
    
    model.add(Dense(l1, activation='relu', input_dim=num_input, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dense(l2, activation='relu', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid', kernel_initializer='he_normal'))

    ## 모델 컴파일
    model.compile(optimizer=adam(lr), loss=losses.binary_crossentropy, metrics=['accuracy', lr_metric])

    # model.summary()
    return model

In [47]:
print([column for column in enumerate(properties)])
svc = SVC(kernel="linear")
# rfe = RFECV(estimator=svc, step = 1, cv=StratifiedKFold(2), scoring="accuracy")
my_model = mlp_model(len(properties),lr=0.001, l1=9, l2=9)
rfecv = RFECV(estimator=DecisionTreeClassifier(), step = 1, cv=StratifiedKFold(5), min_features_to_select = 5, scoring="accuracy")
print(rfecv)
rfecv.fit(X_train,y_train)
## rfe figure plot
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
fig = plt.gcf()
fig.savefig('/home/miruware/Pictures/ResearchDocuments/rfe.png')
plt.show()

## report which features were selected by RFE
for i in range(X_train.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfecv.support_[i], rfecv.ranking_[i]))
    
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# evaluate model

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))    

[(0, 'L/R'), (1, 'age'), (2, 'cT'), (3, 'aAverage'), (4, 'aSD'), (5, 'cAverage'), (6, 'cSD'), (7, 'LR'), (8, 'hu_diff'), (9, 'homogeneous'), (10, 'cN'), (11, 'hetero'), (12, 'rim'), (13, 'clustered'), (14, 'non-mass')]
RFECV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
   estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
   min_features_to_select=5, n_jobs=None, scoring='accuracy', step=1,
   verbose=0)


ValueError: Expected 2D array, got 1D array instead:
array=[0.   0.   0.   0.   0.   0.25 0.   0.75 0.25 0.25 0.   0.25 0.5  0.
 1.   0.   1.   0.   0.25 1.   0.   0.25 0.   0.75 0.25 0.   0.25 0.25
 0.25 0.5  0.   0.5  0.75 0.   0.   0.   0.5  0.   0.25 0.   0.25 0.25
 0.   0.25 0.25 0.75 1.   0.25 0.   0.25 0.25 0.25 0.25 0.25 0.25 0.75
 0.75 0.   0.   0.25 0.25 0.75 0.   0.   0.   0.5  0.25 0.   0.   0.5
 0.   0.25 0.75 0.25 1.   0.75 0.5  0.   0.   0.   0.25 0.   0.   0.5
 0.25 0.25 0.25 0.5  0.   0.   0.25 0.25 0.   0.   0.5  0.   0.25 1.
 0.5  0.   0.25 0.   0.25 0.   0.25 0.75 0.   0.25 0.5  0.75 0.25 0.25
 0.25 0.   0.   0.5  0.5  0.   0.25 0.5  0.   0.   0.   0.5  0.   0.25
 0.25 0.   0.25 0.   0.   0.25 0.   0.5  0.   0.25 0.   0.75 0.   0.25
 0.25 0.5  0.5  0.   0.5  0.25 1.   0.25 0.25 0.   0.25 0.   0.   0.
 0.5  0.25 0.25 0.   0.25 0.   0.25 0.   0.25 0.25 0.25 0.   0.5  0.
 0.   0.   1.   0.   0.   0.   0.25 0.25 0.5  0.25 0.25 0.5  0.5  0.75
 0.25 0.25 0.5  0.25 1.   0.